In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType
from pyspark.sql.functions import lit


spark=SparkSession \
    .builder \
    .appName("FT") \
    .master("local") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/") \
    .config("spark.driver.memory","12g")\
    .config("spark.executor.memory", "12g") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()


:: loading settings :: url = jar:file:/home/dante/spark-3.4.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dante/.ivy2/cache
The jars for the packages stored in: /home/dante/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c33b800e-70ef-4add-9dda-e4e6bcc3796c;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 180ms :: artifacts dl 12ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts  

In [2]:
path = "file:///home/dante/Desktop/Big_data/model_mf"
from pyspark.ml import PipelineModel
# Load the saved model
loaded_model_mf = PipelineModel.load(path)
csv_path = "file:///home/dante/Desktop/Big_data/id.csv"

# Read the CSV file into a DataFrame
schema = StructType([
    StructField("asin", DoubleType(), nullable=True)
])

df = spark.read.csv(csv_path, header=True, schema=schema)

In [4]:
# Prompt the user to enter a double value
user_input = input("Enter a double value: ")

# Convert the user input to a double
value = float(user_input)

final_df = df.withColumn("reviewer_id", lit(value))
df = final_df.select("reviewer_id", "asin")
final_df = df.withColumnRenamed("reviewer_id", "reviewerID_index").withColumnRenamed("asin", "asin_index")
user_predictions = loaded_model_mf.transform(final_df)

In [5]:
from pyspark.sql.functions import desc

# Sort the user_predictions DataFrame by the predictions column in descending order
sorted_predictions = user_predictions.orderBy(desc("prediction"))

# Select the top 5 rows and show the asin_index column
top_5_asin = sorted_predictions.select("asin_index").limit(5)
top_5_asin.show()

23/05/12 15:32:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 14.0
 Schema: asin
Expected: asin but found: 14.0
CSV file: file:///home/dante/Desktop/Big_data/id.csv


+----------+
|asin_index|
+----------+
|     236.0|
|    4288.0|
|   36436.0|
|   40107.0|
|     545.0|
+----------+



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 39722)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/dante/spark-3.4.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/home/dante/spark-3.4.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/home/dante/spark-3.4.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/accumulators.py", line 257, in accu